In [1]:
%pylab
from __future__ import print_function

%matplotlib inline

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function

import csv
from src import main, utils
import sys

projects = main.load_projects()
interests = ['argouml v0.26.2 method', 'mucommander v0.8.5 method']

p = list()
for project in projects:
    if ' '.join([project.name, project.version, project.level]) not in interests:
        continue
    p.append(project)

In [3]:
project = p[1]
print(project)
repos = main.load_repos(project)
goldsets = main.load_goldsets(project)

queries = main.create_queries(project)
snapshot = main.create_release_corpus(project, repos)
changesets = main.create_corpus(project, repos, main.ChangesetCorpus, use_level=False)

snapshot_lda, _  = main.create_lda_model(project, snapshot, None, "Release", use_level=True)
changeset_lda, _ = main.create_lda_model(project, changesets, None, "Changeset", use_level=False)

Project(name='mucommander', printable_name='muCommander', version='v0.8.5', level='method', ref='refs/tags/release_0_8_5', alpha=None, eta=None, passes=5, iterations=1000, num_topics=500, src_url='http://trac.mucommander.com/changeset/3794/tags/release_0_8_5?old_path=%2F&format=zip', data_path='data/mucommander/', full_path='data/mucommander/v0.8.5/', src_path='data/mucommander/v0.8.5/src')


- argouml 5258
    - snapshot 1 org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()
    - changeset 8138 org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()
- mucommander 37
    - snapshot 1 com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame)
    - changeset 303 com.mucommander.ui.action.impl.MinimizeWindowAction.performAction()
- argouml 5088
    - changeset 1 org.argouml.model.AbstractActivityGraphsHelperDecorator.setParameters(Object,Collection)
    - snapshot 124 org.argouml.model.mdr.XmiWriterMDRImpl.write()
- mucommander 142
    - changeset 1 com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics)
    - snapshot 536 com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run()


In [12]:
queries.metadata = True
qs = list(filter(lambda x: x[1][0] in ["37", "142"], queries))

for query, metadata in qs:
    doc = sorted(query, key=lambda x: x[1], reverse=True)
    words = [ ( queries.id2word[wid], freq ) for wid, freq in doc ]
    print("{0}: {1}".format(metadata[0], words))

37: [(u'mac', 3), (u'menu', 3), (u'window', 2), (u'minimize', 2), (u'missing', 2), (u'zoom', 2), (u'application', 1), (u'mimic', 1), (u'options', 1), (u'standard', 1)]
142: [(u'java', 7), (u'drive', 6), (u'icons', 3), (u'button', 3), (u'popup', 3), (u'names', 3), (u'refresh', 3), (u'exception', 2), (u'hashtable', 2), (u'main', 1), (u'lang', 1), (u'mucommander', 1), (u'com', 1), (u'util', 1), (u'npe', 1), (u'pointer', 1), (u'run', 1), (u'thread', 1)]


In [13]:
q = dict()
for query, topics in main.get_topics(snapshot_lda, queries, by_ids=["37", "142"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q[query[0]] = topics
print(q)

for qid, topics in q.items():
    print("{0}: {1}".format(qid, snapshot_lda.print_topic(topics[0][0])))

{u'142': [(486, 0.1925694369028981), (129, 0.15234868830050233), (197, 0.095442557020264801), (433, 0.08724433759517676), (365, 0.086974209650473991), (254, 0.080041283410674041), (75, 0.068508874795707012), (397, 0.048131191819368584), (472, 0.046690501765133535), (168, 0.03295394438506101), (315, 0.031143139357585854), (385, 0.024795605465440455), (133, 0.022039154517503202)], u'37': [(280, 0.29792918451625949), (21, 0.16174993810819474), (437, 0.11122222222222183), (192, 0.11028288903929083), (483, 0.095845374015254059), (31, 0.063825967472166054), (492, 0.053027059006361073), (345, 0.051450698953582036)]}
142: 0.225*java + 0.067*version + 0.041*menu + 0.032*versions + 0.028*item + 0.025*prop + 0.016*param + 0.016*starts + 0.016*read + 0.015*text
37: 0.223*menu + 0.092*item + 0.077*action + 0.056*mnemonic + 0.053*add + 0.045*helper + 0.030*frame + 0.027*toolkit + 0.027*main + 0.027*manager


In [14]:
qc = dict()
for query, topics in main.get_topics(changeset_lda, queries, by_ids=["37", "142"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc[query[0]] = topics
print(qc)

for qid, topics in qc.items():
    print("{0}: {1}".format(qid, changeset_lda.print_topic(topics[0][0])))

{u'142': [(392, 0.55498270127058258), (498, 0.23340548765539929), (230, 0.065519657788999527), (265, 0.063065696966905282), (342, 0.030755349187869461), (425, 0.027950980740096228)], u'37': [(384, 0.23901652345428809), (197, 0.15751591394648801), (69, 0.14623273806620907), (489, 0.10381318908726098), (76, 0.087859094749404398), (89, 0.057612358711653239), (461, 0.055157024925628954), (106, 0.054306300499271432), (13, 0.046802592535192811)]}
142: 0.128*popup + 0.069*drive + 0.068*menu + 0.065*volumes + 0.057*extended + 0.051*name + 0.048*system + 0.029*item + 0.024*mnemonic + 0.023*icons
37: 0.072*dialog + 0.055*rename + 0.054*batch + 0.051*action + 0.038*missing + 0.038*translation + 0.031*label + 0.025*com + 0.025*mucommander + 0.011*recent


In [30]:
g37 = ["com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame)",
       "com.mucommander.ui.action.impl.MinimizeWindowAction.performAction()"]
g142 = ["com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run()",
        "com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics)"]

In [31]:
q37 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g37[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q37[query[0]] = topics
#print(q37)

for qid, topics in q37.items() -
    print("{0}: {1}".format(qid, snapshot_lda.print_topic(topics[0][0])))

com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame): 0.223*menu + 0.092*item + 0.077*action + 0.056*mnemonic + 0.053*add + 0.045*helper + 0.030*frame + 0.027*toolkit + 0.027*main + 0.027*manager


In [32]:
qc37 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g37[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc37[query[0]] = topics
#print(qc37)

for qid, topics in qc37.items():
    print("{0}: {1}".format(qid, changeset_lda.print_topic(topics[0][0])))

com.mucommander.ui.action.impl.MinimizeWindowAction.performAction(): 0.150*window + 0.135*focus + 0.111*frame + 0.109*main + 0.056*request + 0.034*event + 0.031*active + 0.027*current + 0.021*listener + 0.021*activated


In [33]:
q142 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g142[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q142[query[0]] = topics
#print(q142)

for qid, topics in q142.items():
    print("{0}: {1}".format(qid, snapshot_lda.print_topic(topics[0][0])))

com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run(): 0.393*icon + 0.144*file + 0.033*dimension + 0.020*manager + 0.018*icons + 0.018*set + 0.017*name + 0.014*link + 0.014*provider + 0.013*cache


In [34]:
qc142 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g142[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc142[query[0]] = topics
#print(qc142)

for qid, topics in qc142.items():
    print("{0}: {1}".format(qid, changeset_lda.print_topic(topics[0][0])))

com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics): 0.123*space + 0.115*free + 0.071*theme + 0.058*editor + 0.032*extensions + 0.030*dialog + 0.028*warning + 0.022*critical + 0.016*feel + 0.016*threshold
